In [ ]:
from FAdo.fa import *
from FAdo.reex import *
from FAdo.fio import *
from FAdo.rndadfa import *

import lark

In [ ]:
new_idea = NFA()
# ab+ba
sigma = set('ab')
num_states = 2*len(sigma)

new_idea.setSigma(['a','b'])

new_idea.addState("0")
new_idea.addState("1")
new_idea.addState("2")
new_idea.addState("3")

new_idea.addInitial(0)
new_idea.addFinal(3)

new_idea.addTransition(0, 'a', 0)
new_idea.addTransition(0, 'b', 0)
new_idea.addTransition(0, 'a', 1)
new_idea.addTransition(0, 'b', 2)
new_idea.addTransition(1, 'a', 0)
new_idea.addTransition(2, 'b', 0)
new_idea.addTransition(2, 'a', 1)
new_idea.addTransition(1, 'b', 2)
new_idea.addTransition(1, 'b', 3)
new_idea.addTransition(2, 'a', 3)
new_idea.addTransition(3, 'b', 2)
new_idea.addTransition(3, 'a', 1)


In [ ]:
graph = NFA() # abab + baba
word = "ababababababb"
graph.setSigma(set(word))
graph.addState("0")
graph.addState("1")
graph.addState("2")
graph.addState("3")
graph.addState("4")
graph.addState("5")
graph.addState("6")
graph.addState("7")

graph.addInitial(0)
graph.addFinal(7)

# abab
graph.addTransition(0, 'a', 1)
graph.addTransition(1, 'b', 2)
graph.addTransition(2, 'a', 3)
graph.addTransition(3, 'b', 7)

# baba
graph.addTransition(0, 'b', 4)
graph.addTransition(4, 'a', 5)
graph.addTransition(5, 'b', 6)
graph.addTransition(6, 'a', 7)

# inter
graph.addTransition(1, 'b', 4)
graph.addTransition(2, 'a', 5)
graph.addTransition(3, 'b', 6)

graph.addTransition(4, 'a', 1)
graph.addTransition(5, 'b', 2)
graph.addTransition(6, 'a', 3)

graph.addTransition(2, 'a', 1)
graph.addTransition(3, 'b', 2)
graph.addTransition(3, 'b', 4)

graph.addTransition(5, 'b', 4)
graph.addTransition(6, 'a', 5)
graph.addTransition(6, 'b', 1)

graph.addTransitionStar(7, 0)
graph.addTransitionStar(0, 0)

graph.display()


In [ ]:
def cnt(grp, wrd):
	ilist = grp.epsilonClosure(set([0]))
	ctr = 0
	for i in wrd:
		ilist = grp.evalSymbol(ilist, i)
		for s in grp.Final:
			if s in ilist:
				ctr+=1
				break

	return ctr

In [ ]:
cnt(graph, 'ababab')

In [ ]:
def gen_nfa(word):
	kp = word
	sigma = set(kp)

	splt = kp.split("|")
	ng = NFA()
	ng.setSigma(sigma)
	initial_state = ng.addState()
	final_state = ng.addState()
	ng.addInitial(initial_state)
	ng.addFinal(final_state)

	for s in splt:
		ck=0
		cur_init = ng.addState()
		ng.addTransitionStar(initial_state, cur_init)
		last_state = 0
		
		first_symbol_state = None

		letc = 0
		nas = sum([1 for i in s if i == "*"])

		for k in s:
			if k == "*":
				continue

			nl = None
			if letc+1 < len(s):
				nl = s[letc+1]

			if ck == 0:
				last_state = ng.addState()
				first_symbol_state = last_state
				ng.addTransition(initial_state, k, last_state)
			elif ck == len(s)-1-nas:
				ng.addTransition(last_state, k, final_state)
			else:
				old_state = last_state
				last_state = ng.addState()
				ng.addTransition(old_state, k, last_state)
				if nl == "*":
					ng.addTransition(last_state, k, last_state)
			ck+=1
			letc+=1

		for i in ng.States:
			if int(i) != 0:
				for z in sigma:
					if z != s[0]:
						ng.addTransition(int(i), z, 0)
					else:
						ng.addTransition(int(i), s[0], first_symbol_state)
	return ng

In [ ]:
sigma = "ab"
kp = "ab*|ba"
#kp = "aba"

splt = kp.split("|")

auto_list = []
ng = NFA()
ng.setSigma(sigma)
initial_state = ng.addState()
final_state = ng.addState()
ng.addInitial(initial_state)
ng.addFinal(final_state)

for s in splt:
	ck=0
	cur_init = ng.addState()
	ng.addTransitionStar(initial_state, cur_init)
	last_state = 0
	
	first_symbol_state = None

	letc = 0
	nas = sum([1 for i in s if i == "*"])

	for k in s:
		if k == "*":
			continue

		nl = None
		if letc+1 < len(s):
			nl = s[letc+1]

		if ck == 0:
			last_state = ng.addState()
			first_symbol_state = last_state
			ng.addTransition(initial_state, k, last_state)
		elif ck == len(s)-1-nas:
			ng.addTransition(last_state, k, final_state)
		else:
			old_state = last_state
			last_state = ng.addState()
			ng.addTransition(old_state, k, last_state)
			if nl == "*":
				ng.addTransition(last_state, k, last_state)
		ck+=1
		letc+=1

	for i in ng.States:
		if int(i) != 0:
			for z in sigma:
				if z != s[0]:
					ng.addTransition(int(i), z, 0)
				else:
					ng.addTransition(int(i), s[0], first_symbol_state)

ng.display()

In [ ]:
cnt(ng, "abbbbabababbbab")

In [ ]:
xg = ng.toDFA().minimal()
xg.display()

In [ ]:
def dfa_cnt(dfa: DFA, s):
	ls = 0
	c=0
	for i in s:
		ls = dfa.evalSymbol(ls, i)
		if (ls in dfa.Final):
			c=c+1
	return c

dfa_cnt(xg, "abbaabbbb")

In [ ]:
new = gen_nfa("aab*")
imax = new.toDFA().minimal()
#imax.display()

dfa_cnt(imax, "aabbaabbbbbbbb")